In [1]:
# Author : Zane(Ye Wei)
# -*- coding = utf-8 -*-
# @Time ：2022-04-02 16:28
# @Author : 叶 伟
# @File : 平行宇宙算法.py
# @Software : jupyterlab

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import pandas as pd
import os

from numpy import ndarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [3]:
df = pd.read_csv('./dataset/combine/2.storke_combine_code_multi_balance.csv')
Y = np.array(df.pop('NIHSS'))
X = np.array(df)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y,random_state = 3)

In [6]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,auc,roc_curve
from sklearn.preprocessing import LabelBinarizer,StandardScaler

from sklearn import model_selection as cv
from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier

import matplotlib as mpl
import matplotlib.pyplot as plt

from dbn.tensorflow import SupervisedDBNClassification
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.layers import Input,Dense,LSTM,GRU,BatchNormalization
from tensorflow.keras.layers import PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error as MAE
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
import tensorflow.keras as K
import tensorflow as tf

In [8]:
Class = [0,1,2]
Class_dict = dict(zip(Class, range(len(Class))))
Class_dict
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
y_train_labels = lb.transform(y_train)

In [9]:
def Predict(X):
    RNN_test_label = []
    Class = [0,1,2]
    Class_dict = dict(zip(Class, range(len(Class))))
    Class_dict
    for i in range(0,X.shape[0]):
        RNN_test_label.append(Class_dict[np.argmax(X[i])])
    RNN_test_label = np.array(RNN_test_label,dtype = 'int64')
    return RNN_test_label

In [10]:
def buildDNN(layer1,layer2,n_class):
    init = K.initializers.glorot_uniform(seed=1)
    simple_adam = tf.keras.optimizers.Adam()
    model = K.models.Sequential()
    model.add(K.layers.Dense(units=layer1, input_dim=90, kernel_initializer=init, activation='relu'))
    model.add(K.layers.Dense(units=layer2, kernel_initializer=init, activation='relu'))
    model.add(K.layers.Dense(units=n_class, kernel_initializer=init, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=simple_adam, metrics=['accuracy'])
    model.summary()
    return model

In [18]:
# def buildDBN(layer1,layer2,epoca,K=500):
    
#     # cantidad de neuronas ocultas 
#     hidden_layers = []
#     hidden_layers.append( int(layer1))
#     hidden_layers.append( int(layer2))

#     DBN_classifier = SupervisedDBNClassification(hidden_layers_structure = hidden_layers,
#                                                     learning_rate_rbm=0.05,
#                                                     learning_rate=0.1,
#                                                     n_epochs_rbm=epoca,
#                                                     n_iter_backprop=K,
#                                                     batch_size=32,
#                                                     activation_function='relu',
#                                                     dropout_p=0.2)
#     return DBN_classifier

In [19]:
# disdbn = np.zeros((50,50))
# for i in range(1,10):
#     for j in range(1,10):
#         dbn = buildDBN(layer1=i,layer2=j,epoca=20,K=1000)
#         dbn = dbn.fit(x_train, y_train)
#         DBN_test_pred = dbn.predict(x_test)
#         disdbn[i,j]= accuracy_score(y_test, DBN_test_pred)

In [21]:
# for i in range(10,20):
#     for j in range(10,20):
#         dbn = buildDBN(layer1=i,layer2=j,epoca=20,K=1000)
#         dbn = dbn.fit(x_train, y_train)
#         DBN_test_pred = dbn.predict(x_test)
#         disdbn[i,j]= accuracy_score(y_test, DBN_test_pred)

In [22]:
# disdbn

In [23]:
def buildLSTM(timeStep,inputColNum,outStep,i,j,learnRate=1e-4):

    inputLayer = Input(shape=(timeStep,inputColNum))


    middle = LSTM(i,activation='tanh')(inputLayer)
    middle = Dense(j,activation='tanh')(middle)


    outputLayer = Dense(outStep)(middle)
    

    model = Model(inputs=inputLayer,outputs=outputLayer)
    optimizer = Adam(learning_rate=learnRate)
    model.compile(optimizer=optimizer,loss='mse') 
    model.summary()
    return model

x_train_lstm = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
x_test_lstm = x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [26]:
def func(x,y):
    # 目标恒星的能量程度
    acc = dismat[x,y]
    ACC = -acc
    return ACC
# def func(x,y):
#     # 目标恒星的能量程度
#     acc = dbndismat[x,y]
#     ACC = -acc
#     return ACC

# 平行宇宙算法

In [34]:
class PUSE(object):
    def __init__(self, n, narvs, k, x_lb, x_ub):
        self.n = n #恒星数量
        self.narvs = narvs #宇宙的维度
        self.x_lb = x_lb  # 宇宙的边界
        self.x_ub = x_ub # 宇宙的边界
        self.k = k # 膨胀的次数（时间）
        
        self.explore1 = 2 # 个体恒星的膨胀因子
        self.explore2 = 2 # 所有恒星的膨胀因子
        self.w_start = 5 # 恒星的冲击波
        self.w_end = 1 # 恒星的冲击波
        self.volumemax = 6 #最大爆炸能量

        
        self.x1 = [] #经度
        self.x2 = [] #维度
        self.e1 = []
        self.e2 = []
        self.fit = np.zeros((n,1))
                
        self.init_Star()
        
        self.x1 = np.array(self.x1).astype('int64')
        self.x2 = np.array(self.x2).astype('int64')
        
        self.init_Energy()
        
        
        self.xbest = []
        self.prexbest = []

        self.init_MAXenergy()
        
        
 
    def init_Star(self):
        m1 = np.zeros((self.n,1))
        m2 = np.zeros((self.n,1))
        n1 = np.zeros((self.n,1))
        n2 = np.zeros((self.n,1))
        
        m1[0]=0.7      
        for i in range(1,self.n):
            k = i-1
            m1[i] = 2.3*((m1[k])**2)*math.sin(math.pi*m1[k])
        m2[0]=m1[self.n-1]
        for i in range(1,self.n):
            k = i-1
            m2[i] = 2.3*((m2[k])**2)*math.sin(math.pi*m2[k])
            
        for i in range(self.n):        
            n1[i] = int(np.ceil(m1[i]*(self.x_lb-self.x_lb))+self.x_lb) 
            n2[i] = int(np.ceil(m2[i]*(self.x_lb-self.x_lb))+self.x_lb)
            self.x1.append(n1[i])
            self.x2.append(n2[i])
        self.e1 = [int(np.ceil(np.random.rand()*(2*self.volumemax)))-self.volumemax for i in range(self.n)]
        self.e2 = [int(np.ceil(np.random.rand()*(2*self.volumemax)))-self.volumemax for i in range(self.n)]
        
    def init_Energy(self):
        for i in range(self.n):
            self.fit[i] = func(self.x1[i],self.x2[i])
    
    def init_MAXenergy(self):
        fitbest = min(self.fit)
        indbest = self.fit.tolist().index(fitbest)
        self.xbest.append(self.x1[indbest])
        self.xbest.append(self.x2[indbest])
        self.prexbest = self.xbest

    def iterator(self):       
        for d in range(self.k):
            for i in range(self.n):
                w = int(np.ceil(self.w_start - (self.w_start-self.w_end)*d/self.k))
                self.e1[i] = int(np.ceil(w*self.e1[i] + self.explore1*np.random.rand()*(self.prexbest[0]-self.x1[i]) + self.explore2*np.random.rand()*(self.xbest[0]-self.x1[i])))
                self.e2[i] = int(np.ceil(w*self.e2[i] + self.explore1*np.random.rand()*(self.prexbest[1]-self.x2[i]) + self.explore2*np.random.rand()*(self.xbest[1]-self.x2[i])))
                for j in range(self.n):
                    if self.e1[j] < -self.volumemax:
                        self.e1[j] = -self.volumemax
                    elif self.e1[j] > self.volumemax:
                        self.e1[j] = self.volumemax
                    if self.e2[j] < -self.volumemax:
                        self.e2[j] = -self.volumemax
                    elif self.e2[j] > self.volumemax:
                        self.e2[j] = self.volumemax
                self.x1[i] = self.x1[i] + self.e1[i]
                self.x2[i] = self.x2[i] + self.e2[i]
                for j in range(self.n):
                    if self.x1[j] < self.x_lb:
                        self.x1[j] = self.x_lb
                    elif self.x1[j] > self.x_ub-1:
                        self.x1[j] = self.x_ub-1
                    if self.x2[j] < self.x_lb:
                        self.x2[j] = self.x_lb
                    elif self.x2[j] > self.x_ub-1:
                        self.x2[j] = self.x_ub-1 
                self.fit[i] = func(self.x1[i],self.x2[i]) 
                if self.fit[i] < func(self.prexbest[0],self.prexbest[1]):
                    self.prexbest[0] = self.x1[i]
                    self.prexbest[1] = self.x2[i]
                if self.fit[i] < func(self.xbest[0],self.xbest[1]):
                    self.xbest[0] = self.prexbest[0]
                    self.xbest[1] = self.prexbest[1]
                self.fitbest = func(self.xbest[0],self.xbest[1])
        return self.fitbest,self.xbest

In [35]:
if __name__ == '__main__':
    n = 30 #恒星数量
    narvs = 2 #宇宙的维度
    k = 100 # 膨胀的次数
    x_lb = 1  # 宇宙的边界
    x_ub = 100# 宇宙的边界
    my_PUSE = PUSE(n=n, narvs=narvs, k=k, x_lb=x_lb, x_ub=x_ub)
    fitness = my_PUSE.iterator()
    print(fitness)

(array([-0.94680851]), [array([99], dtype=int64), array([70], dtype=int64)])
